<a href="https://colab.research.google.com/github/carlnotsagan/LSST-DSFP-Session15-Materials/blob/main/fields_intro_to_GPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function, division, absolute_import 

# Introduction to GPUs (in Python):

By Carl Fields (Los Alamos National Lab)

*This exercise was designed heavily based on tutorials from the [GTC 2018 Conference](https://github.com/ContinuumIO/gtc2018-numba).* 

---

## Problem 0) Creating Our HPC Environment
**Before** beginning, we want to prepare a new environment for the purpose of this exercise.

Be sure to activate our HPC environment from last time:

```linux
$ conda activate hpc
```
---



## Learning Objectives

- Using Numba decorators to speed up algorithms targeting GPUs
- Creating complex alogrithms utilizing Numba decorators targeting GPUs
- Memory management with _CUDA_ kernels
- Exploring General _CUDA_ kernels
---

We can check that our installation worked by trying to import the required libraries:

In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt

%matplotlib notebook

## Problem 1) Exploring a basic algorithm targeting the GPU


We want to begin by compaaring a native numpy function which will leverage the CPU and compare that to a Ufunc that will target the CPU. 

Lets start by considering the addition of two numbers.

**Problem 1a** Run the following cell to compute the addition of two arrays:

In [3]:
import numpy as np

a = np.array([1, 2, 3, 4])
b = np.array([10, 20, 30, 40])

np.add(a, b)

array([11, 22, 33, 44])

More information on Numpy Ufuncs can be found [here](https://docs.scipy.org/doc/numpy/reference/ufuncs.html).

Next, we want to explore using Numba to create Ufuncs that target the GPU.

**Problem 1b** Use the `vectorize` decorator in Numba to write a function that adds to arrays. Use the `int64` data types and `target='cuda'`. Note: An overview including some common terminology for CUDA programming can be found [here](https://numba.readthedocs.io/en/stable/cuda/overview.html).

In [14]:
from numba import vectorize # complete

@vectorize(['int64(int64, int64)'], target='cuda') # complete
def add_ufunc(x, y):
    return x + y

NvvmSupportError: libNVVM cannot be found. Do `conda install cudatoolkit`:
dlopen(libnvvm.dylib, 0x0006): tried: '/Users/meh/miniforge3/envs/hpc/lib/python3.10/lib-dynload/../../libnvvm.dylib' (no such file), '/Users/meh/miniforge3/envs/hpc/bin/../lib/libnvvm.dylib' (no such file), 'libnvvm.dylib' (no such file), '/usr/local/lib/libnvvm.dylib' (no such file), '/usr/lib/libnvvm.dylib' (no such file), '/Users/meh/Desktop/programs/lsstc_dsfp/LSSTC-DSFP-Sessions/Sessions/Session15/Day5/libnvvm.dylib' (no such file)

Before running we need to be sure we have the hardware we would like to target!

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Check that the GPU was found:

In [15]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

UsageError: Line magic function `%tensorflow_version` not found.


More information can be found about the available types of devices:

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

**Problem 1c** Run your Ufunc which utilizes the GPU and compare the numerical result to Numpy.

In [ ]:
# complete
assert np.allclose(# complete

Now, lets use our favorite `timeit` magic command to see how much we benefitted from targeting the GPU.


**Problem 1d** Use `timeit` to compare the execution time of the default Numpy Ufunc and our new Numba Ufunc

In [ ]:
# complete   # NumPy on CPU

In [ ]:
# complete # Numba on GPU

The GPU result is... *slower*?

**Problem 1e** Discuss in this situation why our GPU result may be slower and how we can modify the problem to benfit fromm the GPU.

**Answer** 

Some points to consider: 

 - **Our inputs are too small**: the GPU achieves performance through parallelism, operating on thousands of values at once. Our test inputs have only 4 and 16 integers, respectively. We need a much larger array to even keep the GPU busy.


- **Our calculation is too simple**: Sending a calculation to the GPU involves quite a bit of overhead compared to calling a function on the CPU. If our calculation does not involve enough math operations (often called "arithmetic intensity"), then the GPU will spend most of its time waiting for data to move around.


- **We copy the data to and from the GPU**: While including the copy time can be realistic for a single function, often we want to run several GPU operations in sequence. In those cases, it makes sense to send data to the GPU and keep it there until all of our processing is complete.


- **Our data types are larger than necessary**: Our example uses int64 when we probably don't need it. 

## Problem 2) Exploring a more complex, data-intensive algorithm targeting the GPU

We will now consider a more complex example where we can take some of the necessary steps to make the problem more efficient to run on GPUs. A few of these steps include: 

1. Using native math module functions described [here](https://docs.python.org/3/library/math.html).
2. Using a less precise datatype than necessary. Consider using `float32` instead. 
3. Solving a more complex algorithm - one with more math operations in this case than the addition of two arrays. 
4. Precompute constant values when possible.


**Problem 2a** Take the above steps to define the Ufunc for a Gaussian PDF using Numba `vectorize` again targeting `cuda`: 

 $f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{1}{2} \left ( \frac{x-\mu}{\sigma} \right )^{2}}$.

 Information on solving Normal distributions are discussed [here](https://en.wikipedia.org/wiki/Normal_distribution).[link text](https://)

In [ ]:
import math  # Note that for the CUDA target, we need to use the scalar functions from the math module, not NumPy

SQRT_2PI = # complete  # Precompute this constant as a float32.  Numba will inline it at compile time.

# complete
def gaussian_pdf(x, mean, sigma):
    '''Compute the value of a Gaussian probability density function at x with given mean and sigma.'''
    return math.exp(-0.5 * ((x - mean) / sigma)**2) / (sigma * SQRT_2PI)

**Problem 2b** Evaluate our Ufunc a million times, set $\mu=0$ and $\sigma=1$. Use `np.random.uniform` to create our `x` array for a bound of [-3,3].: 

In [ ]:
# Evaluate the Gaussian a million times!
x = np.random.uniform# complete
mean = # complete
sigma =# complete

# Quick test
# complete

**Problem 2c** Perform the same calculation using scipys native `norm` function (details [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html)). Time the results using `timeit`.

In [ ]:
import scipy.stats # for definition of gaussian distribution
norm_pdf = # complete
%timeit # complete

**Problem 2d** Time the result for our GPU Ufunc for comparison:

In [ ]:
# complete

**Problem 2e** Thats a big improvement! 

Discuss some of the overhead costs still associated with this approach.

**Answer**: Copying data to and from the GPU.

## Problem 3) Memory Management 

Some key terms before proceeding:

- host: the CPU

- device: the GPU

- host memory: the system main memory

- device memory: onboard memory on a GPU card

- kernels: a GPU function launched by the host and executed on the device

- device function: a GPU function executed on the device which can only be called from the device (i.e. from a kernel or another device function)

In this problem we are going to explore explicit memory management as a way to further increase the speed of our algorithms. 

**Problem 3a** Use the `vectorize` decorator in Numba to write a function that adds two arrays. 

Use the `float32` data types and `target='cuda'`. 

This will serve as the basis of our timing analysis before optimization. 

In [ ]:
# complete
def add_ufunc(x, y):
    return x + y

**Problem 3b** Construct an array `x` of a range of 100000 values. Create an array `y=2*x`.

In [ ]:
n = 100000
x = np.arange(# complete
# complete

**Problem 3c** Use these arrays to run and time (using `timeit` our baseline CUDA function)

In [ ]:
# complete # Baseline performance with host arrays

Next, we want to utilize native Numba tools to copy the arrays we created _to device_ (to the GPU - recall our teriminology cheat sheet from earlier). Details about Memory Management using Numba CUDA is found [here](https://numba.readthedocs.io/en/stable/cuda/memory.html).


**Problem 3d** Use Numba to copy our `x` and `y` arrays to device.

In [ ]:
# complete

**Problem 3e** Call our `add_ufunc` routine using the _on device_ arrays and measure the execution speed. 

In [ ]:
# complete

**Problem 3f** What is the main reason for the speedup observed here?

**Answer:** We removed a key step from the process in our original call of `add_ufunc` as done in `Problem 3c`. We copied the array to device outside of the function call, thus removing the step from the calculation.



With our current optimization, there is still an array created in our call to `add_ufunc` that then needs to be copied _back to host_ (the CPU). We can create the output array ahead of time and set it in our call using _device arrays_.

**Problem 3g** Create an output array on device using Numba.

In [ ]:
# complete  # does not initialize the contents, like np.empty()

**Problem 3h** Finally, call our function passing all input and output device arrays we have created and time the execution. 

In [ ]:
# complete

**Problem 3i** Perform the last step by hand, copy the output of the device array to host and print the first 10 elements to verify the result. 

In [ ]:
# complete

## Problem 4) Writing CUDA Kernels

Some definitions:

A **kernel** function is a GPU function that is meant to be called from CPU code (*). 

It gives it two fundamental characteristics:

- kernels cannot explicitly return a value; all result data must be written to an array passed to the function (if computing a scalar, you will probably pass a one-element array);

- kernels explicitly declare their thread hierarchy when called: i.e. the number of thread blocks and the number of threads per block (note that while a kernel is compiled once, it can be called multiple times with different block sizes or grid sizes).


**Problem 4a** Write a _CUDA_ kernel version of our add function.

Details on the API is available [here]()..

In [ ]:
from numba import cuda
# complete
def add_kernel(x, y, out):
    tx = # complete # this is the unique thread ID within a 1D block
    ty = # complete  # Similarly, this is the unique block ID within the 1D grid

    block_size = # complete  # number of threads per block
    grid_size = # complete    # number of blocks in the grid
    
    start = tx + ty * block_size
    stride = block_size * grid_size

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

**Problem 4b** Call our new _CUDA_ kernal:

In [ ]:
import numpy as np

n = 5000000
x = np.arange(n).astype(np.float32)
y = 2 * x
out = np.empty_like(x)

threads_per_block = 128
blocks_per_grid = 30

add_kernel[blocks_per_grid, threads_per_block]# complete
# complete

**Problem 4c** Simplify our add kernel using `cuda` grid tools:

In [ ]:
@cuda.jit
def add_kernel(x, y, out):
    start = cuda.grid# complete     # 1 = one dimensional thread grid, returns a single value
    stride = cuda.gridsiz# complete # ditto

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

**Problem 4d** As before, create device arrays instead of the implicit copy that takes place when passing NumPy arrays to our GPU kernel:

In [ ]:
# complete
# complete
# complete

**Problem 4e** Perform timing for the kernel with and without the use of device arrays.  

In [ ]:
# complete

The slowest run took 4.29 times longer than the fastest. This could mean that an intermediate result is being cached.
10 loops, best of 5: 29 ms per loop


In [ ]:
# complete

100 loops, best of 5: 5.21 ms per loop


Some comments about Kernel synchronization:


- One extremely important caveat should be mentioned here: CUDA kernel execution is designed to be asynchronous with respect to the host program. This means that the kernel launch `add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device)` returns immediately, allowing the CPU to continue executing while the GPU works in the background. Only host<->device memory copies or an explicit synchronization call will force the CPU to wait until previously queued CUDA kernels are complete.

When you pass host NumPy arrays to a CUDA kernel, Numba has to synchronize on your behalf, but if you pass device arrays, processing will continue. If you launch multiple kernels in sequence without any synchronization in between, they will be queued up to run sequentially by the driver, which is usually what you want. If you want to run multiple kernels on the GPU in parallel (sometimes a good idea, but beware of race conditions!), take a look at CUDA streams.

**Problem 4f** Perform timing for the kernel with and without the use of device arrays and with explicit versus implicit synchronization for memory copies.

In [ ]:
# CPU input/output arrays, implied synchronization for memory copies
# complete

CPU times: user 30.2 ms, sys: 0 ns, total: 30.2 ms
Wall time: 32.2 ms


In [ ]:
# GPU input/output arrays, no synchronization (but force sync before and after)
# complete
# complete
# complete

CPU times: user 0 ns, sys: 573 µs, total: 573 µs
Wall time: 583 µs


In [ ]:
# GPU input/output arrays, include explicit synchronization in timing
# complete
# complete

CPU times: user 2.43 ms, sys: 0 ns, total: 2.43 ms
Wall time: 1.68 ms


**Question:** What is observed? When should we be mindful to sync?

**Answer:** Always be sure to synchronize with the GPU when benchmarking CUDA kernels!


## Problem 5) Matrix multiplication (Optional/Challenge)

**Problem 5a**: Rewrite the following implication of matrix multiplication using shared memory. 

Details on shared memory usage in _CUDA_ is described [here](https://numba.readthedocs.io/en/stable/cuda/memory.html#cuda-shared-memory).

In [ ]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B."""
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp